In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

c:\Users\abhinav bhardwaj\miniconda3\envs\ml\lib\site-packages\tensorflow\python\keras\engine\training_arrays_v1.py:37: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.23.0)
  from scipy.sparse import issparse  # pylint: disable=g-import-not-at-top


In [2]:
train_data_dir = 'dataset'

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

In [4]:
train_generator = train_datagen.flow_from_directory(train_data_dir, batch_size = 20, class_mode = 'binary', target_size = (224, 224))

Found 2829 images belonging to 2 classes.


In [5]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.callbacks import EarlyStopping

# Load InceptionV3 model
base_model = InceptionV3(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Add your custom fully connected layers
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# Create the new model
model = tf.keras.models.Model(base_model.input, x)

# Add EarlyStopping callback to monitor training loss
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

# Compile the model with the callback
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), 
              loss='binary_crossentropy', 
              metrics=['acc'])

# Train the model with early stopping using generator
inception_hist = model.fit(train_generator, 
                           steps_per_epoch=20, 
                           epochs=10, 
                           callbacks=[early_stopping])

# Print the final accuracy
print("Final Accuracy: ", inception_hist.history['acc'][-1])


c:\Users\abhinav bhardwaj\miniconda3\envs\ml\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/10
20/20 [==============================] - 18s 328ms/step - loss: 0.6342 - acc: 0.7700
Epoch 2/10
20/20 [==============================] - 7s 348ms/step - loss: 0.5434 - acc: 0.7815
Epoch 3/10
20/20 [==============================] - 6s 283ms/step - loss: 0.5041 - acc: 0.7975
Epoch 4/10
20/20 [==============================] - 5s 259ms/step - loss: 0.5751 - acc: 0.7550
Epoch 5/10
20/20 [==============================] - 5s 258ms/step - loss: 0.5780 - acc: 0.7650
Epoch 6/10
20/20 [==============================] - 5s 270ms/step - loss: 0.4773 - acc: 0.8025
Epoch 7/10
20/20 [==============================] - 5s 268ms/step - loss: 0.5155 - acc: 0.7750
Epoch 8/10
20/20 [==============================] - 6s 285ms/step - loss: 0.5312 - acc: 0.7700
Epoch 9/10
20/20 [==============================] - 6s 290ms/step - loss: 0.4472 - acc: 0.8050
Epoch 10/10
20/20 [==============================] - 6s 273ms/step - loss: 0.4227 - acc: 0.8325
Final Accuracy:  0.8324999809265137
